In [1]:
from flask import Flask, request
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_excel('TextExcel.xlsx')


# Table preview
preview_html = df.head().to_html(classes='table table-striped', index=False)

# Plot first numeric column
numeric_cols = df.select_dtypes(include='number').columns
date_cols = df.select_dtypes(include=['datetime', 'datetime64']).columns
if not numeric_cols.any():
    chart_html = "<p>No numeric columns to plot.</p>"
else:
    date_col_in = date_cols[0]
    date_col_out = date_cols[1]
    
    
    
    col_in = numeric_cols[0]
    col_out = numeric_cols[1]

    df_in = df[[date_col_in, col_in]].copy()
    df_in['Type'] = 'Purchase'
    df_in.rename(columns={'Time_in':'Time', 'In': 'Quantity'}, inplace=True)

    df_out = df[[date_col_out, col_out]].copy()
    df_out['Type'] = 'Sale'
    df_out.rename(columns={'Time _out':'Time', 'Out':'Quantity'}, inplace=True)

    if df_out['Quantity'].sum() > 0:
        df_out['Quantity'] = df_out['Quantity']*(-1)

    df_combined = pd.concat([df_in, df_out])

    
    df_combined['Time'] = pd.to_datetime(df_combined['Time']).dt.date
    df_combined = df_combined.sort_values(by = 'Time' )
    df_combined.dropna(inplace=True)

    print(df_combined)
    
    
    fig = go.Figure()

  
 
    df_pivot = df_combined.pivot_table(
        index='Time',
        columns='Type',
        values='Quantity',
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    # Sort by time
    df_pivot = df_pivot.sort_values('Time')

    # Create the figure
    fig = go.Figure()

    # Add Purchase bars
    fig.add_trace(go.Bar(
        x=df_pivot['Time'],
        y=df_pivot.get('Purchase', [0]*len(df_pivot)),
        name='Purchase',
        marker_color='green'
    ))

    # Add Sale bars
    fig.add_trace(go.Bar(
        x=df_pivot['Time'],
        y=df_pivot.get('Sale', [0]*len(df_pivot)),
        name='Sale',
        marker_color='red'
    ))

    # Update layout
    fig.update_layout(
        title=f"Bar Chart of '{col_in}'",
        xaxis_title='Time',
        yaxis_title='QTY',
        barmode='group',  # 'group' for side-by-side bars, 'stack' to stack them
        xaxis=dict(type='category')  # Optional: you can remove this if 'Time' is datetime
    )
        


    fig.show()

         Time  Quantity      Type
0  2025-01-01      10.0  Purchase
0  2025-01-02      -2.0      Sale
1  2025-01-03      -8.0      Sale
2  2025-01-05      -3.0      Sale
3  2025-01-07      -5.0      Sale
4  2025-01-18      -8.0      Sale
1  2025-01-20      10.0  Purchase
2  2025-02-17      10.0  Purchase
5  2025-02-20      -2.0      Sale
